In [1]:
import torch
from torch import nn
import numpy as np
import os
import time
import codecs
import re
import json
import pandas as pd
import glob
import datetime
import random

In [2]:
#things to fill out
rootdir = "maddy driscoll/MyFitbitData5/Mad/personal & Account"
regex = re.compile('weight-2021-03-05')
startDate = '03/13/21' 
endDate = '03/28/21'

#get time period
startd = datetime.datetime.strptime(startDate, "%m/%d/%y")
endd = datetime.datetime.strptime(endDate, "%m/%d/%y")
date_array = (startd + datetime.timedelta(days=x) for x in range(0, (endd-startd).days))
dates = []
for date_object in date_array:
    dates = np.append(dates, date_object.strftime("%m/%d/%y"))

    
#get weight files
filenames = os.listdir(rootdir)
weights= []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            weights = np.append(weights, file)
print(weights)            


#get weight data
roughdata = []
for x in range(np.size(weights)):
    filename = rootdir + "/" + weights[x]    
    data = []
    weight = []
    fat = []
    with open(filename) as json_file:
        rawdata = json.load(json_file)
        for n in range(np.size(rawdata)):
            date = rawdata[n]["date"]
            if date in dates:
                roughdata.append(rawdata[n])  
                
        
        for x in range(np.size(roughdata)):
            
            del roughdata[x]["logId"]
            del roughdata[x]["time"]
            del roughdata[x]["source"]
            del roughdata[x]["bmi"]
            weight = np.append(weight, roughdata[x]["weight"])
            fat = np.append(fat, roughdata[x]["fat"])

        data = np.append(data, roughdata)
        



print(fat)

print(weight)


['weight-2021-03-05.json']
[15.30000019 14.6        13.89999962 13.19999981 13.19999981 14.60000038
 14.60000038 14.60000038 13.89999962 13.89999962 13.89999962 14.60000038
 14.60000038 13.19999981 13.19999981]
[114.6 114.6 112.  112.8 112.8 112.  112.2 110.  110.8 109.8 109.8 110.4
 107.6 109.8 108.7]


In [3]:
#things to fill out
rootdir = "maddy driscoll/MyFitbitData5/Mad/Nutrition"
regex = re.compile('food_logs-0')
startD = "2021-03-12"
endD = "2021-03-27"


#get calorie data
filenames = os.listdir(rootdir)
foodfiles = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            foodfiles = np.append(foodfiles, file)

start = False
#get food entries
for x in range(np.size(foodfiles)):
    filename = rootdir + "/" + foodfiles[x]
    data = []
    count = 0
    with open(filename) as json_file:
        roughdata = json.load(json_file)
        for y in range(np.size(roughdata)):
            if roughdata[y-count]["logDate"] == startD:
                start = False
            if roughdata[y-count]["logDate"] == endD:
                start = True
            if start == True:
                cal = roughdata[y-count]["loggedFood"]["calories"]                
                del roughdata[y-count]["logId"]
                del roughdata[y-count]["loggedFood"]
                del roughdata[y-count]["favorite"]
                if "nutritionalValues" in roughdata[y-count].keys():                
                    del roughdata[y-count]["nutritionalValues"]                
                roughdata[y-count]["calories"] = cal
            else:
                del roughdata[y-count]
                count +=1
        
        data = np.append(data, roughdata)
        
        
#get calories for each day
calories = data[0]["calories"]
totCalories = np.empty((0,1))
dates = np.empty((0,1))
for x in range(len(data)-1):
    if data[x+1]["logDate"] == data[x]["logDate"]:
        calories += data[x+1]["calories"]
        if x == len(data)-2:
            totCalories = np.vstack((totCalories, calories))
    else:
        dates = np.vstack((dates, data[x]["logDate"]))
        totCalories = np.vstack((totCalories, calories))
        calories = data[x+1]["calories"]
    

totCalories = np.flip(totCalories)
print(totCalories)

[[1052.]
 [ 932.]
 [ 601.]
 [ 810.]
 [ 775.]
 [ 757.]
 [1021.]
 [2191.]
 [ 658.]
 [ 744.]
 [1097.]
 [ 632.]
 [ 857.]
 [1611.]
 [1261.]]


In [4]:
#calculate calories burned more/less than consumed based on weight change
def caloriesBurned(weight, fat):
    fatMass = []
    leanMass = []
    for x in range(len(weight)):
        fatMass = np.append(fatMass, weight[x]*fat[x]/100)
        leanMass = np.append(leanMass, weight[x] - fatMass[x])
    calories = [] 
    for x in range(len(weight) - 1):   
        calories = np.append(calories, -(fatMass[x]-fatMass[x+1])*442.39)
        if leanMass[x] >= leanMass[x+1]:
            calories[x] -= (leanMass[x] - leanMass[x+1])*70
        else:
            calories[x] -= (leanMass[x]-leanMass[x+1])*265
    calories = np.append(calories, [0])

    return calories
        
#calculate total calories burned in a day
calories = caloriesBurned(weight, fat)
print(calories)
for x in range(len(calories)):
    calories[x] = totCalories[x] - calories[x]

#calories = np.append(calories, [2000,2000, 2000])
print(calories)

[-142.30229677 -615.3131631    91.65866162   -0.          488.58337522
   58.17978814 -273.61167113   95.13531859 -121.76220858   -0.
  360.84063081 -348.23303598  367.29365004 -131.07102722    0.        ]
[1194.30229677 1547.3131631   509.34133838  810.          286.41662478
  698.82021186 1294.61167113 2095.86468141  779.76220858  744.
  736.15936919  980.23303598  489.70634996 1742.07102722 1261.        ]


In [5]:
#things to fill in
rootdir = "maddy driscoll/MyFitbitData5/Mad/Physical Activity"
regex = re.compile('heart_rate-2021-03')
listSplitValue = 2677
#get heart rates

filenames = os.listdir(rootdir)
heartRates = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            heartRates = np.append(heartRates, file)


#read files for hr data
inputs = np.empty((0,3))
dateTime = []
start = False
minInputs = np.empty((0,3))
hrDates = np.empty((0,1))
count = 1
for x in range(np.size(heartRates)):
    filename = rootdir + "/" + heartRates[x]
    data = []
    
    with open(filename) as json_file:
        data = json.load(json_file)
        inputs = np.empty((len(data), 3))
        for y in range(len(data)):
            temp = data[y]["dateTime"][9:] 
            
            
            if data[y]["dateTime"][:8] == startDate:
                start = True
            
                
            if data[y]["dateTime"][:8] == endDate:
                start = False
            if start == True:
                inputs[y][0] = int(temp[:2])*3600  + int(temp[3:5])*60 + int(temp[6:9])
                inputs[y][1] = int(data[y]["value"]["bpm"])
                inputs[y][2] = count
            
       # hrDates = np.vstack((hrDates, data[10]["dateTime"][:8]))
        if start == True:
            count += 1
            
            minInputs = np.vstack((minInputs, inputs))



from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))

for x in range(len(minInputs) - 1):
    if minInputs[x][0] <= minInputs [x+1][0]:
        minInputs[x][0] = minInputs[x+1][0] - minInputs[x][0]
    else:
        minInputs[x][0] = 86400 - minInputs[x][0] + minInputs[x+1][0]
        
        
print(minInputs)    

#normalize, scale and format
minInputs = np.delete(minInputs, len(minInputs)-1, 0)
train_data_normalized2 = scaler.fit_transform(np.transpose(minInputs)[1] .reshape(-1,1))
scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized1 = scaler.fit_transform(np.transpose(minInputs)[0] .reshape(-1,1))

scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized3 = scaler.fit_transform(np.transpose(minInputs)[2] .reshape(-1,1))

train_data_normalized = np.hstack((train_data_normalized1, train_data_normalized2))

train_data_normalized = np.hstack((train_data_normalized, train_data_normalized3))
train_data_normalized =torch.FloatTensor(train_data_normalized)
inputs = train_data_normalized.view(-1, listSplitValue, train_data_normalized.shape[1])
print(inputs)
numOfInputs = np.size(np.transpose(minInputs)[0])

[[1.0000e+01 4.0000e+01 1.0000e+00]
 [5.0000e+00 4.1000e+01 1.0000e+00]
 [1.5000e+01 4.0000e+01 1.0000e+00]
 ...
 [5.0000e+00 5.1000e+01 1.5000e+01]
 [5.0000e+00 5.0000e+01 1.5000e+01]
 [8.6389e+04 5.1000e+01 1.5000e+01]]
tensor([[[-0.9944, -0.9096, -1.0000],
         [-0.9975, -0.8983, -1.0000],
         [-0.9913, -0.9096, -1.0000],
         ...,
         [-0.9913, -0.8870, -1.0000],
         [-0.9944, -0.9209, -1.0000],
         [-0.9975, -0.8983, -1.0000]],

        [[-0.9975, -0.8870, -1.0000],
         [-0.9975, -0.8870, -1.0000],
         [-0.9913, -0.8757, -1.0000],
         ...,
         [-0.9975, -0.1186, -1.0000],
         [-0.9944, -0.1299, -1.0000],
         [-0.9975, -0.1638, -1.0000]],

        [[-0.9975, -0.1525, -1.0000],
         [-0.9944, -0.1412, -1.0000],
         [-0.9975, -0.1751, -1.0000],
         ...,
         [-0.9913, -0.7288, -1.0000],
         [-0.9975, -0.7288, -1.0000],
         [-0.9969, -0.7062, -1.0000]],

        ...,

        [[-1.0000, -0.4011,  1.0

In [6]:
#calculate expected outputs
y_actual = []
day = 0
print("woo")
for x in range(numOfInputs):
    interval = minInputs[x][0]
  
    day = minInputs[x][2]
    
    intervalCalorie = calories[int(day-1)]/86400 * interval        
    y_actual = np.append(y_actual, intervalCalorie)


scaler = MinMaxScaler(feature_range=(-1,1))
print("yay")
normalized_y = torch.FloatTensor(scaler.fit_transform(y_actual .reshape(-1,1)))
print(np.shape(normalized_y))

outputs = normalized_y.view(-1, listSplitValue, normalized_y.shape[1])
print(np.shape(outputs))
print(y_actual)
print(normalized_y)
print(outputs)

woo
yay
torch.Size([168651, 1])
torch.Size([63, 2677, 1])
[0.13822943 0.06911472 0.20734415 ... 0.07297454 0.07297454 0.07297454]
tensor([[-0.9931],
        [-0.9966],
        [-0.9896],
        ...,
        [-0.9964],
        [-0.9964],
        [-0.9964]])
tensor([[[-0.9931],
         [-0.9966],
         [-0.9896],
         ...,
         [-0.9896],
         [-0.9931],
         [-0.9966]],

        [[-0.9966],
         [-0.9966],
         [-0.9896],
         ...,
         [-0.9966],
         [-0.9931],
         [-0.9966]],

        [[-0.9966],
         [-0.9931],
         [-0.9966],
         ...,
         [-0.9896],
         [-0.9966],
         [-0.9959]],

        ...,

        [[-0.9994],
         [-0.9979],
         [-0.9987],
         ...,
         [-0.9927],
         [-0.9964],
         [-0.9927]],

        [[-0.9964],
         [-0.9890],
         [-0.9890],
         ...,
         [-0.9964],
         [-0.9964],
         [-0.9964]],

        [[-0.9964],
         [-0.9890],
        

In [17]:
'''class RNN(nn.Module):
    def __init__(self, input_size = 3, hidden_size = 10, output_size = 1):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, 1)
        self.linear = nn.Linear(hidden_size, output_size)        
        
    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq),1,-1), self.hidden_cell)
        predictions = self.linear(lstm_out)
        return predictions
    
n_hidden = 5
lstm = RNN()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001)
print(lstm)'''


#backpropogate and feedbackward part of rnn
class RNN(nn.Module):
 
    def __init__(self, input_dim=3, hidden_dim=20, batch_size=listSplitValue, output_dim=1,
                    num_layers=1):
        super(RNN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
 
        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
 
        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)
 
    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
 
    def forward(self, input_seq):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input_seq.view(len(input_seq),1,-1))     
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return y_pred
lstm = RNN()

loss_fn = torch.nn.MSELoss(size_average=False)
optimiser = torch.optim.Adam(lstm.parameters(), lr=0.01)
print("Yay")

Yay


C:\Users\peter\.conda\envs\ailab\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [7]:
class MyRNN(nn.Module):
    def __init__(self, input_size = 1, hidden_size = 10, output_size = 1):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.in2hidden = nn.Linear(input_size + hidden_size, hidden_size)
        self.in2output = nn.Linear(input_size + hidden_size, output_size)
    
    def forward(self, x, hidden_state):
        combined = torch.cat((x, hidden_state), 1)
        hidden = torch.sigmoid(self.in2hidden(combined))
        output = self.in2output(combined)
        return output, hidden
    
    def init_hidden(self):
        return nn.init.kaiming_uniform_(torch.empty(1, self.hidden_size))

In [9]:
rnn = MyRNN()
print(rnn)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.01)   # optimize all cnn parameters
loss_fn = torch.nn.MSELoss(size_average=False)

MyRNN(
  (in2hidden): Linear(in_features=11, out_features=10, bias=True)
  (in2output): Linear(in_features=11, out_features=1, bias=True)
)


C:\Users\peter\.conda\envs\ailab\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
epochs = 100

for epoch in range(epochs):
    for x in range(BATCH_SIZE)
    

In [18]:
epochs = 1000
print(np.shape(inputs))
for i in range(epochs):
    for x in range(40): 
        lstm.zero_grad()

        y_pred = lstm(inputs[x])
        
        single_loss = loss_fn(y_pred.float(), outputs[x].view(2677, 1, 1).float())
        loss = loss_fn(y_pred, outputs[x])
      
        # Zero out gradient, else they will accumulate between epochs
        optimiser.zero_grad()

        # Backward pass
        loss.backward()

        # Update parameters
        optimiser.step()
        
print("WOOOOO")
    
    


torch.Size([63, 2677, 3])


C:\Users\peter\.conda\envs\ailab\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([2677, 1])) that is different to the input size (torch.Size([2677, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

In [ ]:
rnn = MyRNN()
print(rnn)

In [ ]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_fn = torch.nn.MSELoss(size_average=False)

In [ ]:
num = 45
y_pred = lstm(inputs[num])
single_loss = loss_function(y_pred.float(), outputs[0].float())
yo = y_pred.detach().numpy()

woo =scaler.inverse_transform(np.transpose(np.transpose(yo)[0]))
print(np.sum(woo))
#print(woo)
outs = scaler.inverse_transform(outputs[num])
print(np.sum(outs))
print(np.hstack((woo,outs)))


In [233]:
# Hyper Parameters
EPOCH = 100       # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = listSplitValue
INPUT_SIZE = 3      # rnn input size / image width
LR = 0.01            # learning rate
DOWNLOAD_MNIST = True   # set to True if haven't download the data

In [240]:

# Data Loader for easy mini-batch return in training
train_loader = torch.utils.data.DataLoader(dataset=inputs,
                                           batch_size=BATCH_SIZE, shuffle=False)

In [249]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=INPUT_SIZE,
            hidden_size=20,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(20, 1)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x.view(len(x),1,-1))   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out)
        return out

In [250]:
rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(3, 20, batch_first=True)
  (out): Linear(in_features=20, out_features=1, bias=True)
)


In [251]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_fn = torch.nn.MSELoss(size_average=False)


In [252]:
#had big issue with tensor shape for outputs - undetected for a while

# training and testing
size = np.size(outputs[0])
print(np.shape(outputs[0].view(listSplitValue, 1, 1)))
for epoch in range(300):
    for x in range(1):        # gives batch data
        output = rnn(inputs[x])                               # rnn output
        loss = loss_fn(output, outputs[x].view(listSplitValue, 1, 1))                   # cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

    print('Epoch: ', epoch, '| train loss: %.4f' % loss.data)
    
for x in range(10):
    print(output[x], " : ", outputs[0][x])


torch.Size([2677, 1, 1])
Epoch:  0 | train loss: 1391.4951
Epoch:  1 | train loss: 1195.2126
Epoch:  2 | train loss: 1003.1124
Epoch:  3 | train loss: 814.1017
Epoch:  4 | train loss: 631.0071
Epoch:  5 | train loss: 458.6605
Epoch:  6 | train loss: 303.3830
Epoch:  7 | train loss: 172.6074
Epoch:  8 | train loss: 74.2973
Epoch:  9 | train loss: 15.7048
Epoch:  10 | train loss: 0.8494
Epoch:  11 | train loss: 26.0911
Epoch:  12 | train loss: 75.5567
Epoch:  13 | train loss: 124.6997
Epoch:  14 | train loss: 154.2218
Epoch:  15 | train loss: 158.0511
Epoch:  16 | train loss: 140.3262
Epoch:  17 | train loss: 109.6635
Epoch:  18 | train loss: 75.0507
Epoch:  19 | train loss: 43.6513
Epoch:  20 | train loss: 19.9369
Epoch:  21 | train loss: 5.6695
Epoch:  22 | train loss: 0.4022
Epoch:  23 | train loss: 2.2349
Epoch:  24 | train loss: 8.5979
Epoch:  25 | train loss: 16.8979
Epoch:  26 | train loss: 24.9454
Epoch:  27 | train loss: 31.1691
Epoch:  28 | train loss: 34.6644
Epoch:  29 | trai

Epoch:  254 | train loss: 0.2561
Epoch:  255 | train loss: 0.2561
Epoch:  256 | train loss: 0.2560
Epoch:  257 | train loss: 0.2560
Epoch:  258 | train loss: 0.2559
Epoch:  259 | train loss: 0.2559
Epoch:  260 | train loss: 0.2558
Epoch:  261 | train loss: 0.2558
Epoch:  262 | train loss: 0.2557
Epoch:  263 | train loss: 0.2557
Epoch:  264 | train loss: 0.2556
Epoch:  265 | train loss: 0.2556
Epoch:  266 | train loss: 0.2556
Epoch:  267 | train loss: 0.2555
Epoch:  268 | train loss: 0.2555
Epoch:  269 | train loss: 0.2554
Epoch:  270 | train loss: 0.2554
Epoch:  271 | train loss: 0.2553
Epoch:  272 | train loss: 0.2553
Epoch:  273 | train loss: 0.2552
Epoch:  274 | train loss: 0.2552
Epoch:  275 | train loss: 0.2551
Epoch:  276 | train loss: 0.2551
Epoch:  277 | train loss: 0.2550
Epoch:  278 | train loss: 0.2550
Epoch:  279 | train loss: 0.2549
Epoch:  280 | train loss: 0.2549
Epoch:  281 | train loss: 0.2548
Epoch:  282 | train loss: 0.2548
Epoch:  283 | train loss: 0.2547
Epoch:  28